# WWTP 예측 모델 학습 노트북

이 노트북은 `scripts/ML/train.py`와 동일한 기능을 제공합니다.

## 사용 가능한 파이프라인
1. **기본 파이프라인**: 여러 ML 모델 비교
2. **개선 파이프라인**: Optuna 하이퍼파라미터 튜닝 + 피처 선택
3. **Sliding Window 파이프라인**: 시계열 패턴 학습

## 모델 모드
- `flow`: 유량 예측 (Q_in)
- `tms`: 전체 수질 예측
- `modelA`: TOC + SS 예측
- `modelB`: TN + TP 예측
- `modelC`: FLUX + PH 예측

## 1. 환경 설정

In [8]:
import sys
from pathlib import Path
import warnings
import os
import numpy as np
import pandas as pd

# 프로젝트 루트 경로 추가
project_root = Path.cwd().parent.parent
sys.path.insert(0, str(project_root))

# 경고 필터
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=UserWarning, module="joblib")

# Windows 환경 최적화
os.environ['LOKY_MAX_CPU_COUNT'] = str(max(1, os.cpu_count() - 1))

print(f"프로젝트 루트: {project_root}")
print(f"Python 버전: {sys.version}")
print(f"사용 가능한 CPU: {os.cpu_count()}")

프로젝트 루트: c:\project_WWTP\python
Python 버전: 3.14.2 | packaged by Anaconda, Inc. | (main, Dec 19 2025, 11:44:48) [MSC v.1929 64 bit (AMD64)]
사용 가능한 CPU: 8


## 2. 라이브러리 임포트

In [ ]:
from src.ML.io import prep_flow, prep_aws, set_datetime_index, merge_sources_on_time
from src.ML.pipeline import run_pipeline, run_improved_pipeline, run_sliding_window_pipeline
from src.ML.features import FeatureConfig
from src.ML.split import SplitConfig
from src.ML.preprocess import ImputationConfig, OutlierConfig
from src.ML.metrics import plot_predictions

print("✅ 모든 모듈 임포트 완료")

✅ 모든 모듈 임포트 완료


## 3. 설정 파라미터

In [ ]:
# ========================================
# 파이프라인 선택
# ========================================
PIPELINE_TYPE = "sliding_window"  # "basic", "improved", "sliding_window"

# ========================================
# 공통 설정
# ========================================
MODE = "flow"  # "flow", "tms", "modelA", "modelB", "modelC"
DATA_ROOT = "data/actual"
RESAMPLE = "1h"  # "5min", "15min", "1h"
RANDOM_STATE = 42

# 데이터 분할 비율
TRAIN_RATIO = 0.7
VALID_RATIO = 0.15
TEST_RATIO = 0.15

# ========================================
# 기본 파이프라인 설정
# ========================================
MERGE_HOW = "outer"  # "inner", "outer", "left", "right"
AGG_METHOD = "mean"  # "mean" 또는 "auto"
PLOT_RESULTS = True  # 결과 시각화 여부

# ========================================
# 개선 파이프라인 설정
# ========================================
N_FEATURES = 50  # 선택할 피처 개수
CV_SPLITS = 3  # 교차 검증 분할 수
N_TRIALS = 50  # Optuna 시도 횟수
SAVE_DIR = "results/ML"  # 결과 저장 디렉토리

# ========================================
# Sliding Window 파이프라인 설정
# ========================================
WINDOW_SIZE = 24  # 과거 24시간 데이터 사용
HORIZON = 1  # 1시간 후 예측
STRIDE = 1  # 매 시간마다 윈도우 생성

# ========================================
# 결측치 처리 설정
# ========================================
USE_ROLLING_MEDIAN = False  # Rolling Median 사용 여부
ROLLING_WINDOW_HOURS = 168  # 7일 (168시간)

print("="*60)
print(f"파이프라인: {PIPELINE_TYPE.upper()}")
print(f"모드: {MODE}")
print(f"리샘플링: {RESAMPLE}")
print("="*60)

파이프라인: IMPROVED
모드: flow
리샘플링: 1h


In [15]:
def load_csvs(path):
    df_flow = pd.read_csv("../../" + path + "/FLOW_Actual.csv")
    df_tms = pd.read_csv("../../" + path + "/TMS_Actual.csv")
    df_aws_368 = pd.read_csv("../../" + path + "/AWS_368.csv")
    df_aws_541 = pd.read_csv("../../" + path + "/AWS_541.csv")
    df_aws_569 = pd.read_csv("../../" + path + "/AWS_569.csv")

    return df_flow, df_tms, df_aws_368, df_aws_541, df_aws_569

## 4. 데이터 로드

In [16]:
print("[1/8] 데이터 로드 중...")

# CSV 파일 로드
df_flow, df_tms, df_aws_368, df_aws_541, df_aws_569 = load_csvs(DATA_ROOT)

# 데이터 전처리
df_flow = prep_flow(df_flow)
df_aws = prep_aws(df_aws_368, df_aws_541, df_aws_569)

# 데이터 딕셔너리 생성
dfs = {"flow": df_flow, "tms": df_tms, "aws": df_aws}
time_col_map = {"flow": "SYS_TIME", "tms": "SYS_TIME", "aws": "datetime"}

print(f"✅ FLOW 데이터: {df_flow.shape}")
print(f"✅ TMS 데이터: {df_tms.shape}")
print(f"✅ AWS 데이터: {df_aws.shape}")

[1/8] 데이터 로드 중...
✅ FLOW 데이터: (130732, 5)
✅ TMS 데이터: (514551, 7)
✅ AWS 데이터: (780744, 28)


## 5. 설정 객체 생성

In [17]:
# 데이터 분할 설정
split_cfg = SplitConfig(
    train_ratio=TRAIN_RATIO,
    valid_ratio=VALID_RATIO,
    test_ratio=TEST_RATIO
)

# 결측치 처리 설정
imputation_cfg = ImputationConfig(
    use_rolling_median=USE_ROLLING_MEDIAN,
    rolling_window_hours=ROLLING_WINDOW_HOURS
)

print("✅ 설정 객체 생성 완료")

✅ 설정 객체 생성 완료


## 6. 파이프라인 실행

In [18]:
if PIPELINE_TYPE == "sliding_window":
    # ========================================
    # Sliding Window 파이프라인
    # ========================================
    print("[3/8] Sliding Window 파이프라인 실행 중...")
    
    out = run_sliding_window_pipeline(
        dfs,
        mode=MODE,
        window_size=WINDOW_SIZE,
        horizon=HORIZON,
        stride=STRIDE,
        time_col_map=time_col_map,
        tz=None,
        resample_rule=RESAMPLE,
        resample_agg="mean",
        split_cfg=split_cfg,
        imputation_cfg=imputation_cfg,
        n_top_features=N_FEATURES,
        cv_splits=CV_SPLITS,
        n_trials=N_TRIALS,
        random_state=RANDOM_STATE,
        save_dir=SAVE_DIR
    )
    
    # 결과 출력
    print("\n" + "="*60)
    print("Sliding Window 정보")
    print("="*60)
    print(f"원본 데이터: {len(out['X_original'])} 샘플")
    print(f"윈도우 생성 후: {len(out['X_seq'])} 윈도우")
    print(f"선택된 피처: {len(out['top_features'])}개")
    print(f"상위 10개: {out['top_features'][:10]}")

elif PIPELINE_TYPE == "improved":
    # ========================================
    # 개선 파이프라인
    # ========================================
    print("[3/8] 개선 파이프라인 실행 중...")
    
    out = run_improved_pipeline(
        dfs,
        mode=MODE,
        time_col_map=time_col_map,
        tz=None,
        resample_rule=RESAMPLE,
        resample_agg="mean",
        split_cfg=split_cfg,
        imputation_cfg=imputation_cfg,
        n_top_features=N_FEATURES,
        cv_splits=CV_SPLITS,
        n_trials=N_TRIALS,
        random_state=RANDOM_STATE,
        save_dir=SAVE_DIR
    )
    
    # 결과 출력
    print("\n" + "="*60)
    print("선택된 피처")
    print("="*60)
    print(f"총 {len(out['top_features'])}개 피처 선택됨")
    print(f"상위 10개: {out['top_features'][:10]}")

else:
    # ========================================
    # 기본 파이프라인
    # ========================================
    print("[2/8] 집계 방법 설정 중...")
    
    if AGG_METHOD == "auto":
        merged_tmp = merge_sources_on_time(
            {k: set_datetime_index(v, time_col_map[k]) for k, v in dfs.items()},
            how=MERGE_HOW
        )
        num_cols = merged_tmp.select_dtypes(include=[np.number]).columns
        resample_agg = {c: ("sum" if str(c).startswith("RN") else "mean") for c in num_cols}
    else:
        resample_agg = AGG_METHOD
    
    print("[3/8] 기본 파이프라인 실행 중...")
    
    out = run_pipeline(
        dfs,
        mode=MODE,
        time_col_map=time_col_map,
        tz=None,
        resample_rule=RESAMPLE,
        resample_agg=resample_agg,
        split_cfg=split_cfg,
        imputation_cfg=imputation_cfg,
        random_state=RANDOM_STATE,
    )
    
    # 결과 출력
    if "period_summary" in out:
        print("\n" + "="*60)
        print("데이터 기간 요약")
        print("="*60)
        print(out["period_summary"].to_string(index=False))
    
    if "continuity" in out:
        print("\n" + "="*60)
        print("연속성 확인")
        print("="*60)
        for key, val in out["continuity"].items():
            print(f"{key}: {val}")

print("\n✅ 파이프라인 실행 완료")

[3/8] 개선 파이프라인 실행 중...

개선된 파이프라인 (Optuna) - 모드: FLOW

[1/9] 시간축 정합 중...

메타데이터 컬럼 제거: ['data_save_dt', 'YYMMDDHHMI_368', 'YYMMDDHHMI_541', 'YYMMDDHHMI_569', 'STN_368', 'STN_541', 'STN_569']

[2/9] 결측치 보간 중 (1차)...

결측치 처리 요약 (1차):
     column  original_missing  ffill  ewma  long_ewma  rolling_median  still_nan
    FLUX_VU            354219   4663   561     348995               0          0
     HM_368             10893   9678     0       1215               0          0
     HM_541             10685    169    60      10456               0          0
     HM_569                88     30     8         50               0          0
      PH_VU            354224   4662   561     349001               0          0
       Q_in            650012    946     4     649062               0          0
 RN_12H_368             17862   9497     0       8365               0          0
 RN_12H_541             24169    240    88      23841               0          0
 RN_12H_569             10795     63   

[W 2026-02-03 17:52:38,868] Trial 7 failed with parameters: {'learning_rate': 0.011240768803005551, 'max_depth': 10, 'min_child_weight': 3, 'subsample': 0.8650089137415928, 'colsample_bytree': 0.7246844304357644, 'gamma': 2.600340105889054} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\user\miniconda3\envs\AIproject\Lib\site-packages\optuna\study\_optimize.py", line 206, in _run_trial
    value_or_values = func(trial)
  File "c:\project_WWTP\python\src\ML\models.py", line 208, in <lambda>
    lambda trial: self._objective(trial, X, y),
                  ~~~~~~~~~~~~~~~^^^^^^^^^^^^^
  File "c:\project_WWTP\python\src\ML\models.py", line 195, in _objective
    scores = cross_val_score(model, X, y, cv=tscv, scoring='neg_mean_squared_error', n_jobs=-1)
  File "c:\Users\user\miniconda3\envs\AIproject\Lib\site-packages\sklearn\utils\_param_validation.py", line 218, in wrapper
    return func(*args, **kwargs)
  File "c:\Users\user\min

KeyboardInterrupt: 

## 7. 결과 확인

In [ ]:
# 성능 지표 테이블
print("\n" + "="*60)
print("모델 성능 비교")
print("="*60)
print(out["metric_table"].to_string(index=False))

# 최고 성능 모델
print("\n" + "="*60)
print("최고 성능 모델")
print("="*60)
best_model_name = out["metric_table"].iloc[0]["model"]
best_r2 = out["metric_table"].iloc[0]["R2_mean"]
best_rmse = out["metric_table"].iloc[0]["RMSE_mean"]
print(f"모델: {best_model_name}")
print(f"Test R²: {best_r2:.4f}")
print(f"Test RMSE: {best_rmse:.2f}")

if PIPELINE_TYPE in ["improved", "sliding_window"]:
    print(f"\n결과 저장 위치: {SAVE_DIR}")

## 8. 예측 결과 시각화 (선택사항)

In [ ]:
if PLOT_RESULTS or PIPELINE_TYPE == "basic":
    print("[8/8] 최고 성능 모델 시각화 중...")
    
    best_model_name = out["metric_table"].iloc[0]["model"]
    best_model = out["fitted_models"][best_model_name]
    X_test, y_test = out["splits"]["test"]
    y_pred = best_model.predict(X_test)
    
    plot_predictions(y_test, y_pred, title=f"TEST | {best_model_name}")
else:
    print("시각화 건너뛰기 (PLOT_RESULTS=False)")

## 9. 추가 분석 (선택사항)

In [ ]:
# 데이터 분할 정보
print("\n" + "="*60)
print("데이터 분할 정보")
print("="*60)
for split_name, (X, y) in out["splits"].items():
    print(f"{split_name.upper()}: X={X.shape}, y={y.shape}")

# 학습된 모델 목록
print("\n" + "="*60)
print("학습된 모델 목록")
print("="*60)
for model_name in out["fitted_models"].keys():
    print(f"  - {model_name}")

## 완료

학습이 완료되었습니다! 결과는 `out` 딕셔너리에 저장되어 있습니다.

### 주요 결과 키
- `out["metric_table"]`: 모델 성능 비교 테이블
- `out["fitted_models"]`: 학습된 모델 딕셔너리
- `out["splits"]`: 데이터 분할 (train, valid, test)
- `out["top_features"]`: 선택된 피처 (개선/슬라이딩 윈도우 파이프라인)
- `out["best_params"]`: 최적 하이퍼파라미터 (개선/슬라이딩 윈도우 파이프라인)